In [3]:
import os
import glob
import pandas as pd

def read_all_files(directory):
    """
    Liest alle Excel-Dateien im Verzeichnis "Indikatoren" ein und gibt ein Dictionary zurück:
       {Dateipfad: DataFrame}
    Der DataFrame wird aus dem Blatt "Transaktionen zum Indiztreffer" erstellt.
    """
    file_pattern = os.path.join(directory, "*.xlsx")
    files = glob.glob(file_pattern)
    data = {}
    for f in files:
        try:
            df = pd.read_excel(f, sheet_name="Transaktionen zum Indiztreffer", engine="openpyxl")
            data[f] = df
        except Exception as e:
            print(f"Fehler beim Lesen der Datei {f}: {e}")
    return data

def add_dup_keys(data):
    """
    Fügt jedem DataFrame im Dictionary 'data' eine neue Spalte 'dup_key' hinzu.
    Die Spalte 'dup_key' wird als Tuple aus den Spalten 
          'Kontonr.', 'Kundennr.', 'Transaktionstyp', 'Buchungstag'
    gebildet und dient als Schlüssel zur Duplikat-Erkennung.
    """
    for df in data.values():
        if 'dup_key' not in df.columns:
            df['dup_key'] = df[['Kontonr.', 'Kundennr.', 'Transaktionstyp', 'Buchungstag']]\
                            .astype(str)\
                            .agg(tuple, axis=1)

def build_global_mapping(data):
    """
    Baut ein globales Mapping: Für jeden 'dup_key' wird als Wert eine Menge erstellt, 
    die alle Dateinamen (vollständige Pfade) enthält, in denen dieser Schlüssel vorkommt.
    Rückgabe: {dup_key: set(Dateinamen)}
    """
    global_mapping = {}
    for filename, df in data.items():
        # Nutze die bereits erzeugte Spalte 'dup_key'
        for key in df['dup_key'].unique():
            if key not in global_mapping:
                global_mapping[key] = set()
            global_mapping[key].add(filename)
    return global_mapping

def analyze_pairwise_duplicates(data, global_mapping):
    """
    Für jeden dup_key, der in mehr als einer Datei vorkommt,
    wird angenommen, dass die Datei mit der niedrigsten Nummer als Original (Quelle) gilt.
    Es werden pairwise-Duplikate gezählt, also:
       {Quelle: {Zieldatei: duplicate_count, ...}, ...}
    """
    def extract_index(fname):
        base = os.path.basename(fname)
        parts = base.split('_')
        try:
            num_str = parts[-1].replace(".xlsx", "")
            return int(num_str)
        except:
            return float('inf')
    
    duplicates = {}
    for key, files in global_mapping.items():
        if len(files) > 1:
            sorted_files = sorted(files, key=extract_index)
            source_file = sorted_files[0]
            for dest_file in sorted_files[1:]:
                # Zähle die Vorkommen von key in der Zieldatei
                dest_count = data[dest_file]['dup_key'].value_counts().get(key, 0)
                duplicates.setdefault(source_file, {}).setdefault(dest_file, 0)
                duplicates[source_file][dest_file] += dest_count
    return duplicates

def compute_global_file_stats(data, global_mapping):
    """
    Berechnet für jede Datei, wie viele Zeilen (Transaktionen) dupliziert sind, 
    also in mindestens einer anderen Datei vorkommen.
    
    Rückgabe ist ein Dictionary:
      { Dateipfad: { 'duplicate_count': X, 'total': Y, 'percentage': Z, 'other_files': [Liste] } }
    """
    file_stats = {}
    for filename, df in data.items():
        total = len(df)
        # Markiere jede Zeile als Duplikat, wenn ihr dup_key in einer anderen Datei vorkommt
        is_dup = df['dup_key'].apply(lambda key: len(global_mapping.get(key, set())) > 1)
        duplicate_count = is_dup.sum()
        # Sammle für diese duplizierten Zeilen alle anderen Dateien, in denen derselbe Schlüssel vorkommt.
        other_files = set()
        for key in df.loc[is_dup, 'dup_key']:
            other_files.update(global_mapping.get(key, set()) - {filename})
        file_stats[filename] = {
            'duplicate_count': duplicate_count,
            'total': total,
            'percentage': (duplicate_count / total * 100) if total > 0 else 0,
            'other_files': sorted(list(other_files))
        }
    return file_stats

def print_pairwise_statistics(duplicates, data):
    """
    Gibt die pairwise Duplikat-Statistiken aus:
    "X Transaktionen aus Datei [Quelle] sind in Datei [Ziel] dupliziert (entsprechen Y% der Transaktionen in Datei [Ziel])."
    """
    print("Pairwise Duplikat-Statistiken:")
    for source, dest_dict in duplicates.items():
        source_name = os.path.basename(source)
        for dest, dup_count in dest_dict.items():
            dest_name = os.path.basename(dest)
            total_dest = len(data[dest])
            percentage = (dup_count / total_dest * 100) if total_dest > 0 else 0
            print(f"{dup_count} Transaktionen aus Datei {source_name} sind in Datei {dest_name} dupliziert "
                  f"(entsprechen {percentage:.2f}% der Transaktionen in Datei {dest_name}).")
    print()

def print_global_file_statistics(file_stats):
    """
    Gibt für jede Datei aus, wie viele ihrer Transaktionen dupliziert sind:
    "In Datei [X] sind Y von Z Transaktionen ({P}%) dupliziert. Diese erscheinen auch in: Datei A, Datei B, ...".
    """
    print("Globale Duplikat-Statistiken pro Datei:")
    for filename, stats in file_stats.items():
        base_name = os.path.basename(filename)
        dup_count = stats['duplicate_count']
        total = stats['total']
        percentage = stats['percentage']
        other_files = [os.path.basename(f) for f in stats['other_files']]
        others_str = ", ".join(other_files) if other_files else "keine"
        print(f"In Datei {base_name} sind {dup_count} von {total} Transaktionen ({percentage:.2f}%) dupliziert. "
              f"Diese erscheinen auch in: {others_str}.")
    print()

def main():
    directory = "Indikatoren"
    if not os.path.exists(directory):
        print(f"Verzeichnis {directory} existiert nicht.")
        return

    data = read_all_files(directory)
    if not data:
        print("Keine Dateien gefunden oder keine Daten zum Verarbeiten.")
        return

    # Ergänze die DataFrames um den Duplikatschlüssel
    add_dup_keys(data)
    # Erstelle ein globales Mapping: dup_key -> Menge der Dateien
    global_mapping = build_global_mapping(data)
    # Berechne pairwise-Duplikate: Quelle -> Ziel -> Anzahl
    duplicates = analyze_pairwise_duplicates(data, global_mapping)
    # Berechne globale Duplikat-Statistiken pro Datei
    file_stats = compute_global_file_stats(data, global_mapping)
    
    # Ausgabe der Statistiken (auf Deutsch)
    print_pairwise_statistics(duplicates, data)
    print_global_file_statistics(file_stats)

if __name__ == "__main__":
    main()

Pairwise Duplikat-Statistiken:
5 Transaktionen aus Datei transactions_1_1.xlsx sind in Datei transactions_1_2.xlsx dupliziert (entsprechen 50.00% der Transaktionen in Datei transactions_1_2.xlsx).
4 Transaktionen aus Datei transactions_1_1.xlsx sind in Datei transactions_1_3.xlsx dupliziert (entsprechen 40.00% der Transaktionen in Datei transactions_1_3.xlsx).
1 Transaktionen aus Datei transactions_1_1.xlsx sind in Datei transactions_1_5.xlsx dupliziert (entsprechen 10.00% der Transaktionen in Datei transactions_1_5.xlsx).
1 Transaktionen aus Datei transactions_1_3.xlsx sind in Datei transactions_1_5.xlsx dupliziert (entsprechen 10.00% der Transaktionen in Datei transactions_1_5.xlsx).

Globale Duplikat-Statistiken pro Datei:
In Datei transactions_1_1.xlsx sind 5 von 10 Transaktionen (50.00%) dupliziert. Diese erscheinen auch in: transactions_1_2.xlsx, transactions_1_3.xlsx, transactions_1_5.xlsx.
In Datei transactions_1_2.xlsx sind 5 von 10 Transaktionen (50.00%) dupliziert. Diese ers